In [84]:
%watermark -mdv

2018-11-19 

CPython 3.7.1
IPython 7.1.1

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 15.6.0
machine    : x86_64
processor  : i386
CPU cores  : 2
interpreter: 64bit


[Documentation](https://docs.python.org/3/reference/datamodel.html#invoking-descriptors)  

# Metaclasses

Some example from https://www.learning-python.com/about-lp5e.html

In [85]:
import requests
from zipfile import ZipFile
from io import BytesIO

# Retrieve zip file from website
zip_url = "https://www.learning-python.com/lp5e-code-1.0-jun1813.zip"
r = requests.get(zip_url, stream=True)

# Get zip and get metaclass examples
z = ZipFile(BytesIO(r.content), mode="r")

examples = [f for f in z.filelist if "metaclass" in f.filename]

## Simple metaclass

In [86]:
def get_code_from_(example_number: int):
    with z.open(examples[example_number]) as myfile:
        print(myfile.read().decode())

In [95]:
from abc import ABC, abstractclassmethod

In [118]:
class MetaOne(type):
    def __prepare__(cls, classname):  # , supers, classdict):
        print('In MetaOne.__prepare__:', cls, classname, sep='\n...')
        return dict()
    def __new__(cls, classname, supers, classdict):
        print('In MetaOne.__new__:', cls, classname, supers, classdict, sep='\n...')
        return super().__new__(cls, classname, supers, classdict)
    def __init__(self, classname, supers, classdict):
        self.print = print
        print('In MetaOne.__init__:', self, classname, supers, classdict, sep='\n...')

class myABC(ABC):
    @abstractclassmethod
    def my_method(self):
        pass

print('making class')
class Spam(metaclass=MetaOne):          # Inherits from none, instance of MetaOne
    data = 1                              # Class data attribute
    def meth(self, arg):                  # Class method attribute
        return self.data + arg
    def __init__(self):
        print('In Spam.__init__:', self, sep='\n...')
    def my_method(self):
        print("My method")



print('making instance')
X = Spam()
print('data:', X.data, X.meth(2))
print(Spam.__bases__, X.__getattribute__, Spam.__mro__)

making class
In MetaOne.__prepare__:
...Spam
...()
In MetaOne.__new__:
...<class '__main__.MetaOne'>
...Spam
...()
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x316d05158>, '__init__': <function Spam.__init__ at 0x316d050d0>, 'my_method': <function Spam.my_method at 0x316d05048>}
In MetaOne.__init__:
...<class '__main__.Spam'>
...Spam
...()
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x316d05158>, '__init__': <function Spam.__init__ at 0x316d050d0>, 'my_method': <function Spam.my_method at 0x316d05048>}
making instance
In Spam.__init__:
...<__main__.Spam object at 0x316d04668>
data: 1 3
(<class 'object'>,) <method-wrapper '__getattribute__' of Spam object at 0x316d04668> (<class '__main__.Spam'>, <class 'object'>)


In [119]:
# https://stackoverflow.com/a/1840466

In [120]:
Spam.print("ouasi")

ouasi


## Example 2

In [75]:
class MetaOne(type):
    def __new__(meta, classname, supers, classdict):
        print('In MetaOne.new:', meta, classname, supers, classdict, sep='\n...')
        _super = supers[0]
#        _super.data = 100
        classdict['data'] = 5
        return type.__new__(meta, classname, supers, classdict)

class Eggs(object):
    data = 10
    pass

print('making class')
class Spam(Eggs, metaclass=MetaOne):                 # Inherits from Eggs, instance of MetaOne
    data = 1                              # Class data attribute
    def meth(self, arg):                  # Class method attribute
        return self.data + arg

print('making instance')
X = Spam()
print('data:', X.data, X.meth(2))

making class
In MetaOne.new:
...<class '__main__.MetaOne'>
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x316dabbf8>}
making instance
data: 5 7


In [76]:
class QSD(Eggs):
    pass

In [77]:
QSD().data

10

In [78]:
Eggs.data

10

In [80]:
Spam.data

5

In [28]:
get_code_from_(1)

from __future__ import print_function

class MetaOne(type):
    def __new__(meta, classname, supers, classdict):
        print('In MetaOne.new:', meta, classname, supers, classdict, sep='\n...')
        return type.__new__(meta, classname, supers, classdict)

class Eggs(object):
    pass

print('making class')
class Spam(Eggs, object):                 # Inherits from Eggs, instance of MetaOne
    __metaclass__ = MetaOne
    data = 1                              # Class data attribute
    def meth(self, arg):                  # Class method attribute
        return self.data + arg

print('making instance')
X = Spam()
print('data:', X.data, X.meth(2))



In [52]:
class A(type):
    def __new__(cls, clsname, bases, methods):
        print(f"Dans A {cls.__mro__}")
        return super().__new__(cls, clsname, bases, methods)

In [53]:
class B(metaclass=A):
    def __new__(cls, arg="default"):
        print(arg)
        return super().__new__(cls)

Dans A (<class '__main__.A'>, <class 'type'>, <class 'object'>)


In [54]:
B(arg="argument")

argument


In [148]:
class B:
    def __new__(cls):
        print(cls.__bases__)
        return super().__new__(cls)
    def aze(self):
        print("B")
    def meth(self):
        print("B")         
class C:
    def __new__(cls):
        return super().__new__(cls)
    def aze(self):
        print("C") 
    def meth(self):
        print("C") 

In [149]:
class A(B, C):

    def __new__(cls, arg="default"):
        print(cls.__mro__)
        if arg == "default":
            return super().__new__(cls)
        else:
            return super().__new__(cls)
    def aze(self):
        print("A")

In [150]:
a = A(arg="default")

(<class '__main__.A'>, <class '__main__.B'>, <class '__main__.C'>, <class 'object'>)
(<class '__main__.B'>, <class '__main__.C'>)


In [151]:
a.aze()
a.meth()

A
B


In [303]:
import itertools
class MyMRO(type): 
    def mro(cls):
        if hasattr(cls, "arg"):
            print(cls.arg)
        print("Enumerating MRO")
        print([b for b in cls.__bases__])
        return tuple([cls] +
                list(itertools.chain.from_iterable(base.__mro__[:-1] for base in cls.__bases__)) +
                [object])
    def __new__(cls, clsname, bases, methods):
        print("Creating my MRO")
        print(cls.__dict__)
        
        return super().__new__(cls, clsname, bases, methods)
class Mixin(metaclass=MyMRO): pass

Creating my MRO
{'__module__': '__main__', 'mro': <function MyMRO.mro at 0x31ec88a60>, '__new__': <staticmethod object at 0x31eda2588>, '__doc__': None}
Enumerating MRO
[<class 'object'>]


In [304]:
class A:
    pass
class B:
    pass

In [305]:
class C(Mixin, A, B):
    def __new__(cls, arg="default"):
        print("Creating C")
        print(cls.__mro__)
        setattr(cls, "arg", arg)
        print(f"in C: {cls.__dict__}")
        return super().__new__(cls)
    def aze(self):
        print("A")

Creating my MRO
{'__module__': '__main__', 'mro': <function MyMRO.mro at 0x31ec88a60>, '__new__': <staticmethod object at 0x31eda2588>, '__doc__': None}
Enumerating MRO
[<class '__main__.Mixin'>, <class '__main__.A'>, <class '__main__.B'>]


In [306]:
C(arg="A")

Creating C
(<class '__main__.C'>, <class '__main__.Mixin'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>)
in C: {'__module__': '__main__', '__new__': <staticmethod object at 0x31eda15c0>, 'aze': <function C.aze at 0x31eccdc80>, '__doc__': None, 'arg': 'A'}


In [254]:
C.arg

'A'

In [240]:
C.__mro__

(__main__.C, __main__.Mixin, __main__.A, __main__.B, object)

In [295]:
class D(A, B):
    def __new__(cls, arg="default"):
        if arg == "A":
            return type('Person', tuple(list(A.__mro__)), dict(A.__dict__))
        else:
            return type('Person', (B) + B.__mro__, dict(B.__dict__))
    def aze(self):
        print("D")

In [297]:
D(arg="A")

AttributeError: type object 'Person' has no attribute 'aze'

# super()

See https://rhettinger.wordpress.com/2011/05/26/super-considered-super/

In [93]:
from collections import OrderedDict

class LoggingDict(dict):
    def __setitem__(self, key, value):
        logging.info('Setting %r to %r' % (key, value))
        super().__setitem__(key, value)
class LoggingOD(LoggingDict, OrderedDict):
    pass

In [92]:
LoggingOD.__mro__

(__main__.LoggingOD,
 __main__.LoggingDict,
 collections.OrderedDict,
 dict,
 object)

The MRO shown above is the one order that follows from those constraints:

- LoggingOD precedes its parents, LoggingDict and OrderedDict
- LoggingDict precedes OrderedDict because LoggingOD.__bases__ is (LoggingDict, OrderedDict)
- LoggingDict precedes its parent which is dict
- OrderedDict precedes its parent which is dict
- dict precedes its parent which is object